In [2]:
from main import Webis17

web17 = Webis17('./data/clickbait17/')
web17.build_corpus(size=19538)




Getting 19538 valid examples from training set.


In [22]:
# print ( web17.corpus[0] )
import numpy as np

def Feature_Tester( tester, data_web ):
    counter = np.zeros( (2,2) )
    for data_i in data_web:
        ground = (int)(data_i[2] >= 0.5) 
        predict = (int)( tester(data_i[0],data_i[1]) )
        counter[ground,predict] += 1
    print ( counter )
    return counter

def tester_1( title, text ):
    return len(title) > 10

def tester_2( title, text ):

    return len(title) > 10



Feature_Tester(tester_1,web17.corpus)
# print ( web17.corpus[:10][2] )


[[1.5000e+01 1.4783e+04]
 [3.0000e+00 4.7370e+03]]


array([[1.5000e+01, 1.4783e+04],
       [3.0000e+00, 4.7370e+03]])

In [23]:
# import torch
# from torch import nn
# from torch.nn.utils.rnn import pack_sequence
# import time
# from torch.utils.data import DataLoader, Dataset
# import gc

# class MyDataset(Dataset):
#     def __init__(self, sen_1, sen_2, simi):
#         self.data1 = torch.from_numpy( np.array(sen_1,dtype=np.float32) )
#         self.data2 = torch.from_numpy( np.array(sen_2,dtype=np.float32) )
#         self.labels = torch.Tensor(simi).type( torch.float32 )
# #         print ( self.labels.size )
    
#     def __getitem__(self,index):
#         sen_1 = self.data1[index] #  torch.DoubleTensor
#         sen_2 = self.data2[index] # torch.DoubleTensor
#         target = self.labels[index]
#         return sen_1,sen_2,target
    
#     def __len__(self):
#         return len(self.labels)


# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         LSTM_hidden_size =  140
#         LSTM_num_layers = 1
#         emb_dim = cur_emb_dim
#         self.LSTM = nn.LSTM(emb_dim,LSTM_hidden_size,LSTM_num_layers, batch_first=True)
# #         self.dense1 = nn.Linear( LSTM_hidden_size,1 )  # set up first FC layer
# #         self.dense2 = nn.Linear( LSTM_hidden_size,1 )  # set up the other FC layer
# #         self.dense = nn.Linear( LSTM_hidden_size,1 )
        
#         Linear_hidden1_size = 40
#         Linear_hidden2_size = 1
        
        
#         self.linear1 = nn.Linear( LSTM_hidden_size,Linear_hidden1_size )
#         self.sigmoid = nn.Tanh(  )
#         self.batchnorm1 = nn.BatchNorm1d( max_length )
# #         self.dropout = nn.Dropout(p=0.2)
#         self.linear2 = nn.Linear( Linear_hidden1_size,Linear_hidden2_size )
        
#         self.cos_sim = nn.CosineSimilarity(dim=1, eps=1e-08)
        
#     def forward(self, input1, input2):
#         lstm1_out, _ = self.LSTM(input1)
#         lstm2_out, _ = self.LSTM(input2)
#         x1 = self.linear1(lstm1_out)
#         x1 = self.sigmoid(x1)
# #         print ( x1.shape )
#         x1 = self.batchnorm1(x1)
# #         x1 = self.dropout(x1)
#         x1 = self.linear2(x1).squeeze()

        
#         x2 = self.linear1(lstm2_out)
#         x2 = self.sigmoid(x2)
#         x2 = self.batchnorm1(x2)
# #         x2 = self.dropout(x2)
#         x2 = self.linear2(x2).squeeze()

# #         x1 = self.dense(lstm1_out).squeeze()
# #         x2 = self.dense(lstm2_out).squeeze()
#         combined = self.cos_sim(x1,x2)
#         out = 1 + 4*combined
#         return out.type( torch.float32 )

# def train(dataloader, model, loss_fn, optimizer):
#     for batch, (X1, X2, y) in enumerate(dataloader):
#         X1, X2, y = X1.to(device), X2.to(device), y.to(device)
# #         if ( batch%3 == 0 ):
# #             pred = model(X2,X1)
# #         else:
#         pred = model(X1,X2)
#         loss = loss_fn(pred, y)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     return model

In [27]:
import torch
from torch import nn
from torch.nn.utils.rnn import pack_sequence
import time
from torch.utils.data import DataLoader, Dataset
import gc

def ArrayWebis( web17_corpus , feature_generator):
    features = []
    labels = []
    for twt in web17_corpus:
        features.append( feature_generator(twt[0],twt[1]) )
        labels.append( twt[2] )
    return features, labels

class MyDataset(Dataset):
    def __init__(self, features, labels):
        self.data = torch.from_numpy( np.array(features,dtype=np.float32) )
        self.labels = torch.from_numpy( np.array(labels,dtype=np.float32) )
    
    def __getitem__(self,index):
        feature = self.data[index]
        label   = self.labels[index]
        return feature, label
    
    def __len__(self):
        return len(self.labels)


class NeuralNetwork(nn.Module):
    def __init__(self, feature_size):
        super(NeuralNetwork, self).__init__()
        Linear_hidden1_size = feature_size
        self.linear1 = nn.Linear( feature_size,Linear_hidden1_size )
        self.active = nn.Tanh( )
        self.linear2 = nn.Linear( Linear_hidden1_size, 1 )
        
    def forward(self, feature, label):
        x = self.linear1( feature )
        x = self.active(x)
        x = self.linear2(x)
        return x.type( torch.float32 )

def train(dataloader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return model

def feature_generator( title , text ):
    return np.array( [len(title), len(text) ] )

gc.collect()
loss_fn = nn.MSELoss()
optimizer = nn.Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

features, labels = ArrayWebis( web17.corpus , feature_generator)

model = NeuralNetwork( len( features[0] ) ).to(device)
print(model)

loss_fn.to(device)

print ( labels[:10] )
batch_size = 150 #, shuffle=True 
train_dataloader = DataLoader( MyDataset(features,labels) ,batch_size=batch_size, shuffle=True) # Collate_fn

print("Data Loaded")

model.train()
epochs = 5
max_score = 0
for itr in range(epochs):
    model.train()
    model = train(train_dataloader, model, loss_fn, optimizer)


def eval_model( model, X, y ):
    model.eval()
    pred_all = []
    with torch.no_grad():
        X = X.to(device)
        y = y.to(device)
        pred = model(X) # .cpu().numpy()
    return stats.pearsonr( X , y )


Using cpu device
NeuralNetwork(
  (linear1): Linear(in_features=2, out_features=2, bias=True)
  (active): Tanh()
  (linear2): Linear(in_features=2, out_features=1, bias=True)
)
[0.13333333332, 1.0, 0.46666666663999995, 0.9333333333199999, 0.0, 0.06666666666, 0.33333333332, 0.06666666666, 0.33333333330000003, 0.13333333332]
Data Loaded


NameError: name 'optimizer' is not defined